# **Classification model**

학습하기에 들어가기 전, 앞서 선택한 최종 변수들을 사용하여 학습/시험용 데이터를 생성하겠습니다.

In [ ]:
x_data = df[['성별코드', '연령대 코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', 
             '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '식전혈당(공복혈당)', '혈색소', 
             '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마 지티피', 
             '흡연상태', '음주여부', '구강검진 수검여부']]
             
y_data = df[['혈압상태']]

7:3의 비율로 학습데이터와 검증데이터를 분리합니다.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=0, stratify = y_data)

scale이 각각 다른 연속형 변수들의 값을 통일시키기 위해 표준화 작업을 수행합니다.

In [ ]:
scaler = StandardScaler()
cols_n = ['신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)', 
          '청력(좌)', '청력(우)', '식전혈당(공복혈당)', '혈색소', '요단백',
          '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마 지티피']
scaler.fit(x_train[cols_n])
x_train[cols_n] = scaler.transform(x_train[cols_n])
x_test[cols_n] = scaler.transform(x_test[cols_n])


Tensorflow와 Pytorch 기반의 심층 신경망 모델을 구축합니다. 

### **Tensorflow**

Dense layer와 은닉층을 증가시켜보았지만, 성능 개선에 도움이 되지 않았습니다. 

    모델 복잡도 > 데이터 복잡도

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units = 128, activation = 'relu'), 
    tf.keras.layers.Dropout(rate = 0.2), 
    tf.keras.layers.Dense(units = 64, activation = 'relu'), 
    # tf.keras.layers.Dropout(rate = 0.2), 
    # tf.keras.layers.Dense(units = 32, activation = 'relu'), 
    # tf.keras.layers.Dropout(rate = 0.2),
    # tf.keras.layers.Dense(units = 3, activation = 'relu'), 
    tf.keras.layers.Dense(units = 3, activation = 'softmax')
])

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
x_train.shape, y_train.shape

((686093, 19), (686093, 1))

In [ ]:
## 조기 종료 옵션

early_stopping = EarlyStopping(
                    monitor='val_accuracy', 
                    patience=5, 
                    verbose=1, 
                    mode='auto', 
                    restore_best_weights=True
)

In [ ]:
## 학습 시작

history = model.fit(x_train, y_train.iloc[:,-1], 
                    validation_data = (x_test, y_test), 
                    batch_size = 50, epochs = 50,
                    callbacks = [early_stopping])

Epoch 1/50
13722/13722 [==============================] - 61s 4ms/step - loss: 1.0199 - accuracy: 0.4936 - val_loss: 1.0147 - val_accuracy: 0.4972
Epoch 2/50
13722/13722 [==============================] - 63s 5ms/step - loss: 1.0147 - accuracy: 0.4975 - val_loss: 1.0141 - val_accuracy: 0.4977
Epoch 3/50
13722/13722 [==============================] - 63s 5ms/step - loss: 1.0137 - accuracy: 0.4977 - val_loss: 1.0149 - val_accuracy: 0.4969
Epoch 4/50
13722/13722 [==============================] - 65s 5ms/step - loss: 1.0133 - accuracy: 0.4985 - val_loss: 1.0128 - val_accuracy: 0.4976
Epoch 5/50
13722/13722 [==============================] - 62s 5ms/step - loss: 1.0129 - accuracy: 0.4983 - val_loss: 1.0138 - val_accuracy: 0.4982
Epoch 6/50
13722/13722 [==============================] - 53s 4ms/step - loss: 1.0126 - accuracy: 0.4990 - val_loss: 1.0134 - val_accuracy: 0.4980
Epoch 7/50
13722/13722 [==============================] - 53s 4ms/step - loss: 1.0125 - accuracy: 0.4988 - val_loss: 1

In [ ]:
# 검증 결과 

model.evaluate(x_test, y_test)

9189/9189 [==============================] - 24s 3ms/step - loss: 1.0120 - accuracy: 0.4983


[1.011994481086731, 0.49831655621528625]